# IBM Applied Data Science Capstone
This notebook will be used for the data science capstone course.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis pd.set_option('display.max_columns', None) pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if not imported
from geopy.geocoders import Nominatim # convert an address into latitude and longit ude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes #uncomment this line if you have n't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Part 1: Downloading data into df

In [3]:
pip install -U virtualenv

     |████████████████████████████████| 7.1MB 5.6MB/s eta 0:00:01██████████████▌     | 5.9MB 5.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/83/71a2ee6158bb9f39a90c0dea1637f81d5eef866e188e1971a1b1ab01a35a/filelock-3.0.12-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/0a/490fa011d699bb5a5f3a0cf57de82237f52a6db9d40f33c53b2736c9a1f9/distlib-0.3.1-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
!pip3 install beautifulsoup4
!pip install beautifulsoup4

     |████████████████████████████████| 115 kB 11.1 MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/66/25/ff030e2437265616a1e9b25ccc864e0371a0bc3adb7c5a404fd661c6f4f6/beautifulsoup4-4.9.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/8f/457f4a5390eeae1cc3aeab89deb7724c965be841ffca6cfca9197482e470/soupsieve-2.0.1-py3-none-any.whl


In [5]:
!pip3 install lxml 
!pip install lxml 
#then restart kernel

     |████████████████████████████████| 5.4 MB 5.3 MB/s eta 0:00:01             | 2.7 MB 5.3 MB/s eta 0:00:01  | 4.1 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 6.7MB/s eta 0:00:01��██████████████████████████████▉| 5.5MB 6.7MB/s eta 0:00:01


In [ ]:
#from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0, flavor='html5lib')[0] #read_html produces a list of dataframes; get desired table by index [0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#drop NA boroughs
NA = df[df['Borough'] == 'Not assigned'].index
    
#delete row indexes from df
df.drop(NA, inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
#change NA neighborhoods to = borough
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
df.shape

(103, 3)

In [14]:
# Export df to csv in case need to retrieve
df.to_csv('Toronto df Part 1.csv',index=False)

## Part 2: Geospatial

In [ ]:
#Adding geospatial data
geo = 'https://cocl.us/Geospatial_data'
toronto_geo= pd.read_csv(geo)

In [ ]:
toronto_geo.set_index("Postal Code")
toronto_geo.head()

In [28]:
df.set_index("Postal Code")
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [30]:
#join geospatial with original df

toronto_full = df.merge(toronto_geo)
toronto_full.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [32]:
# Export full df to csv in case need to retrieve
toronto_full.to_csv('Toronto df full Part 2.csv',index=False)

## Part 3: Clustering

In [3]:
# Re-import part 2 dataframe
part2_csv = "Toronto df full Part 2.csv"
toronto_full = pd.read_csv(part2_csv)
toronto_full.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [4]:
#get long/lat of location
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto latitude: {}, longitude: {}.'.format(latitude, longitude))

Toronto latitude: 43.6534817, longitude: -79.3839347.


### Create map of Toronto

In [5]:
#map of Toronto with Folium
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add markers
for lat, lng, borough, neighborhood in zip(toronto_full['Latitude'], toronto_full['Longitude'], toronto_full['Borough'], toronto_full['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  

map_toronto

### Setup Foursquare API


In [6]:
#Setup Foursquare API
CLIENT_ID = '5BV4TK25OW0TZJTAHWYPXWQ3TE2NIFCFRXNFTLGWQJQKCM5J' # Foursquare ID 
CLIENT_SECRET = 'BQCEZOEZMLPKZQWX1R53EQGPTBMAISWY2XKMSDYXE2DBKFYU' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentials:') 
print('CLIENT_ID: ' + CLIENT_ID) 
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 5BV4TK25OW0TZJTAHWYPXWQ3TE2NIFCFRXNFTLGWQJQKCM5J
CLIENT_SECRET:BQCEZOEZMLPKZQWX1R53EQGPTBMAISWY2XKMSDYXE2DBKFYU


### Exploring a neighbourhood

In [7]:
#Look at any neighborhood - picked the 5th on list
toronto_full.loc[4, 'Neighbourhood']

"Queen's Park, Ontario Provincial Government"

In [8]:
#Get neighbourhood's long/lat values
neighborhood_latitude = toronto_full.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_full.loc[4, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_full.loc[4, 'Neighbourhood'] # neighborhood name 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Latitude and longitude values of Queen's Park, Ontario Provincial Government are 43.6623015, -79.3894938.


In [9]:
#Get top 100 venues in Queen's Park within rad = 600m
LIMIT = 100 #limit of number of venues returned by Foursquare API
radius = 600 #define radius
#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url #display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5BV4TK25OW0TZJTAHWYPXWQ3TE2NIFCFRXNFTLGWQJQKCM5J&client_secret=BQCEZOEZMLPKZQWX1R53EQGPTBMAISWY2XKMSDYXE2DBKFYU&v=20180605&ll=43.6623015,-79.3894938&radius=600&limit=100'

In [ ]:
#send GET request
results = requests.get(url).json() 
results

In [12]:
#need info from items keys
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [13]:
#clean JSON and put into pandas df
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Nando's,Portuguese Restaurant,43.661728,-79.386391
2,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
3,Central YMCA,Distribution Center,43.663083,-79.385025
4,Bar Volo,Beer Bar,43.665462,-79.385692


In [14]:
#number returned by fourquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

57 venues were returned by Foursquare.


In [15]:
#create function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude',
                             'Neighborhood Longitude', 
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category'] 
    return(nearby_venues)

### Running same code for all of Toronto

In [16]:
#code to run above fxn on each neighborhood & create new df for Toronto venues
toronto_venues = getNearbyVenues(names=toronto_full['Neighbourhood'],
                                   latitudes=toronto_full['Latitude'],
                                   longitudes=toronto_full['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [17]:
#size of resulting df
print(toronto_venues.shape)
toronto_venues.head()

(2131, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [18]:
#venues for each neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4
Woodbine Heights,7,7,7,7,7,7


In [19]:
# unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


### Analyze each neighbourhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1 ])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#size of onehot df
toronto_onehot.shape


(2131, 269)

In [22]:
#group rows by neighborhood and by taking the mean of the freq of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Put most common venues into pandas df

In [23]:
#sort venues in descending order
num_top_venues = 10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
#new df for top 10

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Dim Sum Restaurant,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Pizza Place,Supermarket,Sushi Restaurant,Deli / Bodega,Shopping Mall,Restaurant,Mobile Phone Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
4,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Butcher,Sushi Restaurant,Pizza Place,Pharmacy,Indian Restaurant


### Clustering neighborhoods

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_full

# merge toronto_grouped with toronto_full to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

# check the last columns!
toronto_merged.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,BBQ Joint,Food & Drink Shop,Bus Stop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Financial or Legal Service,Intersection,Hockey Arena,French Restaurant,Coffee Shop,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Ice Cream Shop,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Women's Store,Dog Run,Dim Sum Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Diner,Distribution Center,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop


In [33]:
#drop NaN

toronto_merged.dropna()
toronto_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
toronto_merged


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,BBQ Joint,Food & Drink Shop,Bus Stop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Financial or Legal Service,Intersection,Hockey Arena,French Restaurant,Coffee Shop,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Ice Cream Shop,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Women's Store,Dog Run,Dim Sum Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Diner,Distribution Center,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0,River,Pool,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dance Studio
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Pub,Bubble Tea Shop,Yoga Studio
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2.0,Light Rail Station,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Skate Park,Garden,Auto Workshop
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,4.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Farmers Market


In [ ]:
#Turn Cluster Labels into type int
#toronto_merged.dropna()

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)


In [58]:
#Visualize clusters

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters individually

In [97]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,0.0,Pizza Place,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
63,York,0.0,Brewery,Convenience Store,Pizza Place,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
70,Etobicoke,0.0,Pizza Place,Chinese Restaurant,Middle Eastern Restaurant,Coffee Shop,Intersection,Sandwich Place,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant
93,Etobicoke,0.0,Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Dim Sum Restaurant,Deli / Bodega,Department Store


In [98]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,1.0,Park,Pool,Women's Store,Golf Course,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
35,East York,1.0,Park,Convenience Store,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
52,North York,1.0,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
64,York,1.0,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
66,North York,1.0,Park,Convenience Store,Electronics Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
85,Scarborough,1.0,Park,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
91,Downtown Toronto,1.0,Park,Playground,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Electronics Store,Donut Shop,Doner Restaurant,Dance Studio


In [99]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,BBQ Joint,Food & Drink Shop,Bus Stop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,North York,2.0,Portuguese Restaurant,Financial or Legal Service,Intersection,Hockey Arena,French Restaurant,Coffee Shop,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant
2,Downtown Toronto,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Ice Cream Shop,Beer Store
3,North York,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Women's Store,Dog Run,Dim Sum Restaurant
4,Downtown Toronto,2.0,Coffee Shop,Diner,Distribution Center,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,2.0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Asian Restaurant
98,Etobicoke,2.0,River,Pool,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dance Studio
99,Downtown Toronto,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Pub,Bubble Tea Shop,Yoga Studio
100,East Toronto,2.0,Light Rail Station,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Skate Park,Garden,Auto Workshop


In [100]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3.0,Martial Arts School,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [101]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Farmers Market
101,Etobicoke,4.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Farmers Market


## Let's also look for the number of hotels in each area.

In [40]:
#pull list of top hotels in each neighborhood

categoryId = '4bf58dd8d48988d1fa931735'

#A search for venues  "hotels"
search_query = 'hotels'
radius = 500
print(search_query + ' .... OK!')

hotels .... OK!


Let's test our code with one neighborhood first.

In [41]:
#Looking at one neighborhood
url_hotel1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, #from queen's park one above
    neighborhood_longitude, 
    VERSION, 
    categoryId,
    radius, 
    LIMIT)
url_hotel1

'https://api.foursquare.com/v2/venues/search?client_id=5BV4TK25OW0TZJTAHWYPXWQ3TE2NIFCFRXNFTLGWQJQKCM5J&client_secret=BQCEZOEZMLPKZQWX1R53EQGPTBMAISWY2XKMSDYXE2DBKFYU&ll=43.6623015,-79.3894938&v=20180605&categoryId=4bf58dd8d48988d1fa931735&radius=500&limit=100'

In [42]:
results2 = requests.get(url_hotel1).json()['response']['venues']
results2

[{'id': '5ac43562f0b4905d46625a6a',
  'name': 'Toronto',
  'location': {'lat': 43.660533,
   'lng': -79.387507,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.660533,
     'lng': -79.387507}],
   'distance': 253,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto ON', 'Canada']},
  'categories': [{'id': '56aa371be4b08b9a8d5734e1',
    'name': 'Vacation Rental',
    'pluralName': 'Vacation Rentals',
    'shortName': 'Vacation Rental',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hostel_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1597076128',
  'hasPerk': False},
 {'id': '5079e566e4b03bba60a03e4c',
  'name': 'Canada Suites Toronto',
  'location': {'address': '736 Bay Street',
   'crossStreet': 'College Street',
   'lat': 43.65912066508785,
   'lng': -79.38556541860137,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65912066508785,
     'lng': -79.3855654186

In [43]:
#create function to repeat the same process to all the neighborhoods in Toronto

def getNearbyHotels(names, latitudes, longitudes, radius=500):
    hotels_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url_hotels = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            categoryId,
            radius,
            LIMIT)
        
        # make the GET request
        #results = requests.get(url).json()
        results3 = requests.get(url_hotels).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        hotels_list.append([(
            name,
            lat,
            lng,
            v['name'],
            #v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results3])
            v['location']['lat'], v['location']['lng'], v['categories'][0]['name']) for v in results3])

    nearby_hotels = pd.DataFrame([item for hotel_list in hotels_list for item in hotel_list])
    nearby_hotels.columns = ['Neighborhood', 
                             'Neighborhood Latitude',
                             'Neighborhood Longitude', 
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category'] 
    return(nearby_hotels)

In [44]:
#code to run above fxn on each neighborhood & create new df for Toronto venues
toronto_hotels = getNearbyHotels(names=toronto_full['Neighbourhood'],
                                   latitudes=toronto_full['Latitude'],
                                   longitudes=toronto_full['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [45]:
#size of resulting df
print(toronto_hotels.shape)
toronto_hotels.head()

(396, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Residence & Conference Centre,43.653040,-79.357040,Hotel
1,"Regent Park, Harbourfront",43.654260,-79.360636,Locarno Hostel,43.658627,-79.365400,Hostel
2,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Cottages,43.654332,-79.358005,Bed & Breakfast
3,"Regent Park, Harbourfront",43.654260,-79.360636,Red Lion Inn,43.652245,-79.363126,Hotel
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Toronto,43.660533,-79.387507,Vacation Rental


In [46]:
#number hotels for each neighbourhood
toronto_hotels_sum = toronto_hotels.groupby('Neighborhood').count()
toronto_hotels_sum

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Berczy Park,8,8,8,8,8,8
"Brockton, Parkdale Village, Exhibition Place",2,2,2,2,2,2
Caledonia-Fairbanks,1,1,1,1,1,1
Canada Post Gateway Processing Centre,4,4,4,4,4,4
Central Bay Street,21,21,21,21,21,21
Christie,2,2,2,2,2,2
Church and Wellesley,21,21,21,21,21,21
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2


Let's drop all columns except one to keep the total hotel venue numbers.

In [47]:
toronto_hotels_clean = toronto_hotels_sum.drop(['Neighborhood Latitude', 'Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)

toronto_hotels_clean.rename(columns = {'Venue':'Number of Category: Hotels'}, inplace = True)
#toronto_hotels_clean.rename(columns = {'Neighborhood':'Neighbourhood'}, inplace = True)
toronto_hotels_clean


,Number of Category: Hotels
Neighborhood,
"Bathurst Manor, Wilson Heights, Downsview North",1
Berczy Park,8
"Brockton, Parkdale Village, Exhibition Place",2
Caledonia-Fairbanks,1
Canada Post Gateway Processing Centre,4
Central Bay Street,21
Christie,2
Church and Wellesley,21
"Cliffside, Cliffcrest, Scarborough Village West",2


In [94]:
# Export hotel df to csv in case need to retrieve
toronto_hotels_clean.to_csv('Toronto Hotels',index=True)

Now, let's add this as part of the legend in the map above.

## Final map (clusters and hotel numbers)

First, we can join the hotel numbers data with the toronto_merged dataframe.

In [48]:
#only if not done above already 

#toronto_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True) 
#toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,BBQ Joint,Food & Drink Shop,Bus Stop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Financial or Legal Service,Intersection,Hockey Arena,French Restaurant,Coffee Shop,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Ice Cream Shop,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Women's Store,Dog Run,Dim Sum Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Diner,Distribution Center,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0,River,Pool,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dance Studio
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Pub,Bubble Tea Shop,Yoga Studio
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2.0,Light Rail Station,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Skate Park,Garden,Auto Workshop
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,4.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Farmers Market


In [53]:
toronto_final = pd.merge(toronto_merged, toronto_hotels_clean, on='Neighborhood')
toronto_final.rename(columns = {'Number of Category: Hotels':'Hotels'}, inplace = True)

toronto_final #last column is the number of hotels in the region

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Hotels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Ice Cream Shop,Beer Store,4
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Diner,Distribution Center,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,10
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Pizza Place,Theater,Lingerie Store,41
3,M4C,East York,Woodbine Heights,43.695344,-79.318389,2.0,Skating Rink,Park,Pharmacy,Beer Store,Bus Stop,Curling Ice,Colombian Restaurant,Department Store,College Gym,College Rec Center,1
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,Coffee Shop,Café,Clothing Store,Restaurant,Cocktail Bar,American Restaurant,Cosmetics Shop,Gym,Breakfast Spot,Park,27
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2.0,Coffee Shop,Café,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Seafood Restaurant,Gym,8
6,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1.0,Park,Pool,Women's Store,Golf Course,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,1
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2.0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Japanese Restaurant,Miscellaneous Shop,21
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2.0,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Baby Store,Restaurant,Nightclub,Coffee Shop,2
9,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,2.0,Coffee Shop,Bank,Middle Eastern Restaurant,Pizza Place,Supermarket,Sushi Restaurant,Deli / Bodega,Shopping Mall,Restaurant,Mobile Phone Shop,1


In [56]:
# add hotel numbers to map popup

map_final = map_clusters

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors2 = []
for lat, lon, poi, cluster, hotels in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighborhood'], toronto_final['Cluster Labels'], toronto_final['Hotels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ", Hotels: " + str(hotels), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_final)
       
map_final